C:\Users\swati\AppData\Local\Temp\ipykernel_18596\2091636423.py:38: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  resampled['open'] = data['open'].resample(timeframe).first()
C:\Users\swati\AppData\Local\Temp\ipykernel_18596\2091636423.py:39: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  resampled['high'] = data['high'].resample(timeframe).max()
C:\Users\swati\AppData\Local\Temp\ipykernel_18596\2091636423.py:40: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  resampled['low'] = data['low'].resample(timeframe).min()
C:\Users\swati\AppData\Local\Temp\ipykernel_18596\2091636423.py:41: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  resampled['close'] = data['close'].resample(timeframe).last()
C:\Users\swati\AppData\Local\Temp\ipykernel_18596\2091636423.py:42: FutureWarning: 'm

In [ ]:
# Import libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# --- Step 1: Take user inputs ---
symbol = input("Enter Stock Symbol (like SBIN.NS, RELIANCE.NS): ")
start_date = input("Enter Start Date (YYYY-MM-DD): ")
end_date = input("Enter End Date (YYYY-MM-DD): ")
timeframe = input("Enter Timeframe (like 1m, 5m, 15m, 1h, 1d): ")

try:
    # --- Step 2: Check if intraday timeframe is valid ---
    today = datetime.now()
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")

    if timeframe[-1] in ['m', 'h']:
        # Intraday data only allowed for last 60 days
        if start_dt < today - timedelta(days=60):
            print("\n Intraday data (1m, 5m, 15m, 1h) is only available for last 60 days.")
            print("Please use daily data (1d) or choose dates within last 60 days.")
            exit()

    # --- Step 3: Download stock data ---
    print("\nDownloading data... please wait...")
    data = yf.download(symbol, start=start_date, end=end_date, interval=timeframe)

    if data.empty:
        print("No data found for this symbol or date range. Please check inputs.")
        exit()

    # --- Step 4: Clean and format data ---
    data = data.dropna()
    data.reset_index(inplace=True)
    data['date'] = data['Date'].dt.date
    data['time'] = data['Date'].dt.time
    data = data[['date', 'time', 'Open', 'High', 'Low', 'Close', 'Volume']]
    data.columns = ['date', 'time', 'open', 'high', 'low', 'close', 'volume']

    # --- Step 5: Resample if intraday ---
    if timeframe[-1] in ['m', 'h']:
        data['datetime'] = pd.to_datetime(data['date'].astype(str) + ' ' + data['time'].astype(str))
        data.set_index('datetime', inplace=True)

        resampled = pd.DataFrame()
        resampled['open'] = data['open'].resample(timeframe).first()
        resampled['high'] = data['high'].resample(timeframe).max()
        resampled['low'] = data['low'].resample(timeframe).min()
        resampled['close'] = data['close'].resample(timeframe).last()
        resampled['volume'] = data['volume'].resample(timeframe).sum()

        resampled.reset_index(inplace=True)
        resampled['date'] = resampled['datetime'].dt.date
        resampled['time'] = resampled['datetime'].dt.time
        resampled = resampled[['date', 'time', 'open', 'high', 'low', 'close', 'volume']]
        final_data = resampled
    else:
        final_data = data  # Daily data, no resampling needed

    # --- Step 6: Save to CSV ---
    filename = f"{symbol}_{timeframe}.csv"
    final_data.to_csv(filename, index=False)
    print(f"\n✅ Data successfully saved as {filename}")

    print("\nPreview of saved data:")
    preview = pd.read_csv(filename)
    print(preview.head())  #show first 5 rows

except Exception as e:
    print("Error:", e)
    print("Please check your inputs and try again.")


Enter Stock Symbol (like SBIN.NS, RELIANCE.NS):  RELIANCE.NS
Enter Start Date (YYYY-MM-DD):  2025-01-01
Enter End Date (YYYY-MM-DD):  2025-01-20
Enter Timeframe (like 1m, 5m, 15m, 1h, 1d):  1d


C:\Users\swati\AppData\Local\Temp\ipykernel_18348\1145993276.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start_date, end=end_date, interval=timeframe)
[*********************100%***********************]  1 of 1 completed


✅ Data successfully saved as RELIANCE.NS_1d.csv

Preview of saved data:
         date      time         open         high          low        close  \
0  2025-01-01  00:00:00  1210.017281  1221.421805  1206.780209  1216.391846   
1  2025-01-02  00:00:00  1216.391808  1239.499468  1215.146780  1236.860107   
2  2025-01-03  00:00:00  1238.951814  1257.029638  1230.585205  1246.172974   
3  2025-01-06  00:00:00  1248.961727  1256.979753  1210.166719  1213.154785   
4  2025-01-07  00:00:00  1217.138829  1239.549323  1216.391812  1235.913818   

     volume  
0   5892590  
1  15486276  
2  15521102  
3  14816766  
4  10070505  
